In [1]:
import os
import time
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

In [2]:
# Build rm dict 
rm_df = pd.read_excel('D:/Thesis_data/Rm.xlsx')
rm_df['year'] = rm_df['Names Date'].map(lambda x: str(x)[:4])
rm_df['month'] = rm_df['Names Date'].map(lambda x: str(x)[4:6])
rm_dict = {}
for i in range(2005,2019,1):
    rm_dict[i]={}
for _, row in rm_df.iterrows():
    rm_dict[int(row['year'])][int(row['month'])] = row['Value-Weighted Return-incl. dividends']
    
# Build rf dict 
rf_df = pd.read_excel('D:/Thesis_data/Rf.xlsx')
rf_df['year'] = rf_df['Date (SAS). Last Trading Day of the Month'].map(lambda x: str(x)[:4])
rf_df['month'] = rf_df['Date (SAS). Last Trading Day of the Month'].map(lambda x: str(x)[4:6])
rf_dict = {}
for i in range(2005,2020,1):
    rf_dict[i]={}
for _, row in rf_df.iterrows():
    rf_dict[int(row['year'])][int(row['month'])] = row['Risk-Free Return Rate (One Month Treasury Bill Rate)']

# Read Data
df = pd.read_excel('D:/Thesis_data/factorData.xlsx')
df = df.drop('Unnamed: 0', axis=1).reset_index(drop=True)
df = df.drop(df[df['R']=='C'].index)
df['size mark'] = 0
df['book mark'] = 0
df['btm'] = df['B'] / df['price']


In [5]:
final_result_list = []
for year in range(2006,2019,1):
    if year == 2006:
        start_month = 7
    else:
        start_month = 1
    year_df = df[df['year'] == year]
    for month in range(start_month,13,1):
        current_df = year_df[year_df['month'] == month]
        current_df = current_df[current_df['B'] >= 0]

        # Assign by market size
        current_df = current_df.sort_values('market size',ascending=False)
        total_market_size = current_df['market size'].sum()

        current_market_size = 0
        result_list = []
        for _, row in current_df.iterrows():
            current_market_size = current_market_size + row['market size']
            if current_market_size < (0.5*total_market_size):
                row['size mark'] = 'B'
            else:
                row['size mark'] = 'S'
            result_list.append(row)
        result_df = pd.DataFrame(result_list)

        # Assign btm
        result_df = result_df.sort_values('btm')
        n_company = result_df.shape[0]
        current_n_company = 0
        result_list=[]
        for _, row in result_df.iterrows():
            current_n_company = current_n_company + 1
            if current_n_company < (n_company*0.3):
                row['book mark'] = 'L'
            elif current_n_company < (n_company*0.7):
                row['book mark'] = 'M'
            else:
                row['book mark'] = 'H'

            result_list.append(row)
        final_result_list.append(pd.DataFrame(result_list))
df_assigned = pd.concat(final_result_list).sort_index()

In [10]:
df_assigned['mark'] = df_assigned['size mark'] + df_assigned['book mark']
df_assigned['market*R'] = df_assigned['market size'] * df_assigned['R']

In [24]:
portfolio_R_dict = {}
for year in range(2006,2019,1):
    portfolio_R_dict[year] = {}
    if year == 2006:
        start_month = 7
    else:
        start_month = 1
    year_df = df_assigned[df_assigned['year'] == year]
    for month in range(start_month,13,1):
        portfolio_R_dict[year][month]={}
        month_df = year_df[year_df['month'] == month]
        for mark in ['SL', 'SM', 'SH', 'BL', 'BM', 'BH']:
            current_df = month_df[month_df['mark'] == mark]
            R = current_df['market*R'].sum() / current_df['market size'].sum()
            portfolio_R_dict[year][month][mark] = R

            

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [26]:
Rsize_dict = {}
Rbtm_dict = {}
for year in range(2006,2019,1):
    Rsize_dict[year] = {}
    Rbtm_dict[year] = {}
    if year == 2006:
        start_month = 7
    else:
        start_month = 1
    for month in range(start_month,13,1):
        current_dict = portfolio_R_dict[year][month]
        Rsize = (current_dict['SL']+current_dict['SM']+current_dict['SH']
                 -current_dict['BL']-current_dict['BM']-current_dict['BM'])/3
        Rbtm = (current_dict['SH']+current_dict['BM']-current_dict['SL']-current_dict['BL'])/2
        Rsize_dict[year][month] = Rsize
        Rbtm_dict[year][month] = Rbtm

In [30]:
with open('D:/Thesis_data/Rsize_dict.pickle', 'wb') as f:
    pickle.dump(Rsize_dict, f)
with open('D:/Thesis_data/Rbtm_dict.pickle', 'wb') as f:
    pickle.dump(Rbtm_dict, f)

# Calculate Tone

In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
df = pd.read_pickle('D:/Thesis_data/df_final200107.pickle')

In [3]:
df['tone mark'] = 0
result_list = []
for year in range(2006,2019,1):
    if year == 2006:
        start_month = 7
    else:
        start_month = 1
    year_df = df[df['year'] == year]
    for month in range(start_month,13,1):
        month_df = year_df[year_df['month'] == month]
        na_df = month_df[month_df['vader'].isna()]
        current_df = month_df[month_df['vader'].notnull()]
        current_df = current_df.sort_values('vader')
        total_market_size = current_df['market size'].sum()
        current_size = 0
        for _, row in current_df.iterrows():
            current_size += row['market size']
            if current_size < 0.3*total_market_size:
                row['tone mark'] = 'H'
            elif current_size < 0.7*total_market_size:
                row['tone mark'] = 'M'
            else:
                row['tone mark'] = 'L'
            result_list.append(row)
df = pd.DataFrame(result_list)

In [5]:
df['size tone mark'] = df['size mark'] + df['tone mark']

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
def build_btm_size_dict(df):
    portfolio_R_dict = {}
    for year in range(2006,2019,1):
        portfolio_R_dict[year] = {}
        if year == 2006:
            start_month = 7
        else:
            start_month = 1
        year_df = df[df['year'] == year]
        for month in range(start_month,13,1):
            portfolio_R_dict[year][month]={}
            month_df = year_df[year_df['month'] == month]
            portfolio_R_dict[year][month]['Rm'] = month_df['market*R'].sum() / month_df['market size'].sum()
            for mark in ['SL', 'SM', 'SH', 'BL', 'BM', 'BH']:
                current_df = month_df[month_df['mark_in'] == mark]
                R = current_df['market*R'].sum() / current_df['market size'].sum()
                portfolio_R_dict[year][month][mark] = R
    return portfolio_R_dict

In [4]:
tone_dict = {}
for year in range(2006,2019,1):
    tone_dict[year] = {}
    if year == 2006:
        start_month = 7
    else:
        start_month = 1
    year_df = df[df['year'] == year]
    for month in range(start_month,13,1):
        month_df = year_df[year_df['month'] == month]
        high_df = month_df[month_df['tone mark'] == 'H']
        low_df = month_df[month_df['tone mark'] == 'L']
        R_high = high_df['market*R'].sum() / high_df['market size'].sum()
        R_low = low_df['market*R'].sum() / low_df['market size'].sum()
        tone_dict[year][month] = R_high - R_low

In [28]:
tone_dict

{2006: {7: 0.01531222486630705,
  8: 0.001591569789271427,
  9: 0.00017503628150131872,
  10: -0.017858828080079096,
  11: -0.010629968571523152,
  12: 0.01967483020733409},
 2007: {1: -0.016675428377087992,
  2: -0.009801968799226417,
  3: -0.00953243966972199,
  4: -0.0031322141511832616,
  5: -0.006454666370654651,
  6: 0.002109529892692783,
  7: -0.02085700279344109,
  8: 0.0031514075789732497,
  9: -0.016546257740006787,
  10: -0.05783205236019126,
  11: -0.004335607462737125,
  12: -0.004779312305790141},
 2008: {1: 0.04283776456012551,
  2: -0.0036343612272631658,
  3: -0.02591398839488799,
  4: 0.03618756080690105,
  5: -0.011182331308445035,
  6: -0.0027515432166991677,
  7: -0.004445473043497732,
  8: -0.013223960061838267,
  9: 0.03948383622478539,
  10: 0.031063513245776442,
  11: 0.030694535277503363,
  12: -0.004574766874704702},
 2009: {1: -0.03542739931045969,
  2: 0.02769629109147105,
  3: -0.011732312880399637,
  4: -0.029870208176753524,
  5: -0.015283833130764243,
 

In [7]:
import pickle
with open('D:/Thesis_data/tone_dict200107.pickle', 'wb') as f:
    pickle.dump(tone_dict, f)

In [5]:
tone_dict

{2006: {7: -0.019470479708986802,
  8: -0.005159367881014711,
  9: -0.021043305484849666,
  10: -0.005929728810253096,
  11: 0.022474565541520816,
  12: -0.012419734278207868},
 2007: {1: -0.0085358737791995,
  2: 0.017442189179977825,
  3: -0.0007643037261426174,
  4: -0.0014510113973092778,
  5: 0.01259250627549311,
  6: 0.01918101318577304,
  7: -0.017323238150380033,
  8: -0.009464513809180529,
  9: -0.0011594919829391107,
  10: -0.03032457671996558,
  11: -0.019858785885037755,
  12: 0.01819346567729648},
 2008: {1: -0.003211837180741167,
  2: 0.026407929966675032,
  3: -0.045246395562894594,
  4: 0.06512700323461479,
  5: 0.04051047942868437,
  6: 0.009053600866545101,
  7: -0.03353750791838798,
  8: -0.022173890446476337,
  9: -0.05740428061448655,
  10: -0.0353547348537587,
  11: -0.030673684373992406,
  12: 0.024803682764067338},
 2009: {1: 0.01684548377624604,
  2: -0.0145257654288405,
  3: 0.022534907806497556,
  4: 0.03158406116177767,
  5: -0.004562762817270194,
  6: -0.04